In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from keras.src.ops import dtype

train_dir='/home/anaconda/data/signData'
#train_dir='D:/signData'
json_folder_path = f'{train_dir}/train/label/landmark'
test=np.empty((0, 3)) 
for i in range(1,21):
    arr=np.array([i,i,i])
    test=np.append(test,[arr],axis=0)



def transShape(array):
    xl_coords=array[:,0]
    yl_coords=array[:,1]
    zl_coords=array[:,2]
    xl_array=xl_coords.reshape(5,4)
    yl_array=yl_coords.reshape(5,4)
    zl_array=zl_coords.reshape(5,4)
    
    return np.stack([xl_array.T[::-1],yl_array.T[::-1],zl_array.T[::-1]])
        


for person in os.listdir(json_folder_path):
    for word in os.listdir(os.path.join(json_folder_path, person)):
        if "F" in word:
            for frame in os.listdir(os.path.join(json_folder_path, person, word)):
                file_path = os.path.join(json_folder_path,person,word,frame)
                
                preFrameCoordP = np.empty((0, 3))  # Initialize as empty 2D array for 3D coordinates
                preFrameCoordL = np.empty((0, 3))  # For left hand
                preFrameCoordR = np.empty((0, 3))  # For right hand
                frameCoordP = np.empty((0, 3))
                
                
                # Open and read the JSON file
                with open(file_path, 'r') as json_file:
                    try:
                        data = json.load(json_file)  # Parse JSON content
                        lh_points = data['people']['hand_left_keypoints_3d']
                        rh_points = data['people']['hand_right_keypoints_3d']
                        p_points = data['people']['pose_keypoints_3d']
                
                        # Process pose keypoints
                        for j in range(0, len(p_points), 4):
                            xp = 960 *p_points[j] +960   # x-coordinate
                            yp = 1080*p_points[j + 1] + 540    # y-coordinate
                            zp = (p_points[32+2]-p_points[j + 2] ) /10              # z-coordinate
                            arrayP = np.array([xp, yp, zp],dtype=np.float32)
                            
                            # Append to preFrameCoordP
                            preFrameCoordP = np.append(preFrameCoordP, [arrayP], axis=0)
                        
                        # Process left and right hand keypoints
                        for i in range(4, len(lh_points), 4):
                            xl = 960 * lh_points[i] +960   # x-coordinate for left hand
                            yl = 1080 * lh_points[i + 1] +540   # y-coordinate for left hand
                            n_xl=(xl-420)/(1500-420)#normalized x-coordinate for left hand for cut image 1080x1080
                            n_yl=(yl-0)/(1080-0)            
                            zl = (lh_points[2] - lh_points[i + 2]) / 10  # z-coordinate for left hand
                            
                            arrayL = np.array([n_xl, n_yl, zl],dtype=np.float32)
                            preFrameCoordL = np.append(preFrameCoordL, [arrayL], axis=0)
                
                            xr = 960 * rh_points[i] +960  # x-coordinate for right hand
                            yr = 1080 * rh_points[i + 1] +540   # y-coordinate for right hand
                            n_xr=(xr-420)/(1500-420)#normalized x-coordinate for left hand for cut image 1080x1080
                            n_yr=(yr-0)/(1080-0)  
                            zr = (rh_points[2] - rh_points[i + 2]) / 10  # z-coordinate for right hand
                            
                            arrayR = np.array([n_xr, n_yr, zr],dtype=np.float32)
                            preFrameCoordR = np.append(preFrameCoordR, [arrayR], axis=0)
                        
                        for i in range(0,19):
                            frameCoordP=np.append(frameCoordP,[preFrameCoordP[i]], axis=0)
                        

                        frameCoordL=transShape(preFrameCoordL)
                        frameCoordR=transShape(preFrameCoordR)
                        
                    except json.JSONDecodeError as e:
                        print(f"Error reading {file_path}: {e}")
                
                        break


In [1]:
import numpy as np
test=np.empty((0, 3)) 
for i in range(1,21):
    arr=np.array([i,i,i])
    test=np.append(test,[arr],axis=0)
print(test)

[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]
 [ 4.  4.  4.]
 [ 5.  5.  5.]
 [ 6.  6.  6.]
 [ 7.  7.  7.]
 [ 8.  8.  8.]
 [ 9.  9.  9.]
 [10. 10. 10.]
 [11. 11. 11.]
 [12. 12. 12.]
 [13. 13. 13.]
 [14. 14. 14.]
 [15. 15. 15.]
 [16. 16. 16.]
 [17. 17. 17.]
 [18. 18. 18.]
 [19. 19. 19.]
 [20. 20. 20.]]
